In [257]:
import pandas as pd
import gensim.downloader as api
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from sklearn.decomposition import PCA
import numpy as np

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lolab\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [237]:
wv_google_300d = api.load('word2vec-google-news-300')
wv_twitter_50d = api.load('glove-twitter-50')


[=====---------------------------------------------] 11.8% 23.5/199.5MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[===============-----------------------------------] 31.6% 63.0/199.5MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[=========================-------------------------] 51.1% 102.0/199.5MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[===================================---------------] 71.6% 142.8/199.5MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[=============================================-----] 91.9% 183.4/199.5MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [178]:
history_df = pd.read_csv('history_predictions.csv')

incorrect_predictions = history_df.loc[history_df['correct'] == 0]
correct_predictions = history_df.loc[history_df['correct'] == 1]

stopword_list = stopwords.words('english')

stopword_list.append("-B-")
stopword_list.append("-A-")
stopword_list.append("<unk>")

stemmer = nltk.stem.porter.PorterStemmer()

incorrect_predictions['decoded_segment']

9       ['-B-', 'okay', '.', 'i', "'m", 'done', 'anymo...
23      ['-B-', 'i', 'have', 'the', 'same', 'picture',...
60      ['-B-', 'guy', 'on', 'white', 'bike', '-A-', '...
64      ['-B-', 'do', 'you', 'have', 'the', 'lady', 's...
71      ['-B-', 'kk', 'little', 'girl', 'helping', 'da...
                              ...                        
2548    ['-B-', 'teenager', 'with', 'child', '-A-', 'b...
2553     ['-B-', 'plaid', 'apron', '-A-', '<unk>', 'guy']
2560    ['-A-', 'i', 'have', 'a', 'woman', 'with', 'a'...
2571    ['-A-', 'black', 'shirt', 'red', 'lettering', ...
2579    ['-B-', 'laptop', 'on', 'black', 'stand', 'and...
Name: decoded_segment, Length: 263, dtype: object

In [238]:
vectors = []

# incorrect_predictions['decoded_segment'], decoded

wrong_pred = incorrect_predictions['decoded_segment']
good_pred = correct_predictions['decoded_segment']

all_predictions = [good_pred, wrong_pred]


for prediction in all_predictions:

    vector = 0
    i = 0
    
    for sentence in prediction:
        for word in sentence.split(","):

            word = word.replace("'", "")
            word = word.replace(" ", "")
            word = word.lower()

    #         if word is stopword, skip the word
            if word in stopword_list:
                continue

            word = stemmer.stem(word)

            try:

                vector += wv_twitter_50d[word]
                i += 1

            except:
                None
                
    vector = vector / i
    vectors.append(vector)


print(vectors)


[array([-8.92093033e-02, -2.45292753e-01,  8.20635110e-02, -3.35994102e-02,
       -8.69635418e-02, -8.36643483e-03,  5.47086298e-01,  1.65184528e-01,
        2.08654866e-01,  3.48535359e-01, -9.35202986e-02,  1.79868266e-01,
       -3.28301764e+00,  2.66943052e-02, -1.21933129e-02,  1.85036898e-01,
        9.95144770e-02, -3.34673285e-01, -1.51989043e-01, -1.23649687e-01,
       -2.20028803e-01,  8.54320708e-04,  1.69558764e-01, -1.01894453e-01,
        1.08210780e-02,  2.18606386e-02, -1.87982783e-01, -6.04968891e-02,
        8.27350989e-02, -1.98729858e-01,  4.09521982e-02,  3.18558961e-01,
        6.50063828e-02, -3.03752840e-01,  3.01999271e-01, -2.26621963e-02,
       -9.62538645e-03,  2.74224356e-02,  6.34580925e-02, -1.15649833e-03,
       -8.75078917e-01,  9.59849805e-02, -1.70730188e-01,  1.04005583e-01,
        2.70360529e-01, -1.33330375e-02,  3.16587865e-01,  3.78532968e-02,
       -8.57795961e-03,  2.58686971e-02], dtype=float32), array([-7.7965915e-02, -2.4658704e-01,  8

In [271]:
a = vectors[0].reshape(-1, 1)
b = vectors[1].reshape(-1, 1)
# print(a.shape)
a = a.mean()
b = b.mean()

# print(type(a))

# a = np.array([1, 4, 5, 765432, 5, 3])
# a = a.reshape(-1, 1)

pca = PCA(n_components=0.50, whiten = True)
result = pca.fit_transform(a)
result

# result
# pca = PCA(n_components=2)
# # pca.fit(vectors)
# # PCA(n_components=2)


# result = pca.fit(vectors)

# pca.explained_variance_ratio

ValueError: Expected 2D array, got scalar array instead:
array=-0.05679996311664581.
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [251]:
a = [1, 2, 20000]
b = [4, 2, 5434543]

pca = PCA(n_components=2)
pca.fit_transform(a,b )

ValueError: Expected 2D array, got 1D array instead:
array=[1.e+00 2.e+00 2.e+04].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.